In [1]:
# from keras.models import load_model
import tensorflow as tf
import cv2
import numpy as np

In [2]:
model = tf.keras.models.load_model('modelMaskFinal.h5') # Loaded model after so much of struggle lol
#model = tf.keras.models.load_model('model-008.model')

In [3]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source = cv2.VideoCapture(0)

labels_dict = {0:'MASK',1:'NO MASK'}
color_dict = {0:(0,255,0),1:(0,0,255)} # If mask there, bounding rectl. green, else red

In [4]:
# Taken from my image processing SIH repo
while(True): 
    # Read image from webcam frame by frame every second
    ret,img = source.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img = gray[y:y+w,x:x+w] # Region of interest crop
        resized = cv2.resize(face_img,(100,100)) # Resize to match training data
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,100,100,1)) # 4D vector reshaping to feed into NN
        result = model.predict(reshaped) # Predict using the model from CNNTraining script

        label = np.argmax(result,axis=1)[0] # Get prob of mask from result list
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key = cv2.waitKey(1)
    
    if(key==27): #Escape to quit
        break
        
cv2.destroyAllWindows()
source.release()